In [1]:
from Bio import SeqIO
from Bio import Seq
import pandas as pd
import numpy as np
import csv
import re
import os


In [2]:
def seqToString(motif):
    """
    motif: a Seq that represents the motif 
    
    Returns the String representation of the motif
    """
    i = 0
    string = ""
    length = motif.__len__()
    while i < length:
        string += motif.__getitem__(i)
        i += 1
    return string

In [3]:
def getNegative(pos_seq):
    """
    pos_seq = dna sequence in the positive direction reading from the file
    
    Returns the negative counterpart of the positive sequence.
    """
    dict = {"A":'T','T':'A','G':'C','C':'G','-':'-','N':'N'}
    negative = ""
    last_index = len(pos_seq) - 1
    while last_index > -1:
        negative += dict[pos_seq[last_index].upper()]
        last_index -= 1
    return negative

In [4]:
def getMotifLength(motif_key):
    dir = "/Users/niharikadesaraju/motif_extraction/data/jaspar_fm/modified/"
    name = motif_key + ".jaspar"
    for file in os.listdir(dir):
        if file.endswith(name):
            file_dir = dir + file
            f = open(file_dir, 'r')
            length = 0
            for line in f:
                a = line.split()
                length = len(a)
                return length

In [46]:
def prep_table(spec, motif_key, no_thresh_path):
    """ 
        spec: The number representing a region ex: 11048
        motif_key: The three character code representing a motif
        path: The path to the folder with the thresholded and non thresholded directories
        
        Returns table containing the columns: strand direction, aligned position, score, species id, raw postion,
        and whether or not the position is considered the beginning of a motif
        
    """
    print("is no thresh path")
    print(os.path.isfile(no_thresh_path))
    print(no_thresh_path)

    no_thresh = pd.read_csv(no_thresh_path)
    thresh = no_thresh.loc[no_thresh["score"] >= 7]
    verif_motifs = thresh.drop(columns = ['score', 'motif', 'raw_position', 'Unnamed: 0', 'strand'])
    keys = verif_motifs['species'].to_list()
    vals = verif_motifs['align_position'].to_list()
    verif_motifs_dict = dict(zip(keys, vals))
    thresh = thresh.drop_duplicates(subset = 'align_position')
    thresh = thresh.drop(columns = ['score', 'motif', 'raw_position', 'Unnamed: 0'])
    no_thresh = no_thresh.drop(columns = ['motif', 'Unnamed: 0'])
    result = pd.merge(thresh, no_thresh, on = ['align_position', 'strand'])
    result.drop(columns = ['species_x'])
    orig = []
    for i in np.arange(len(result)):
        orig += ['no']
    for key in verif_motifs_dict:
        species = result['species_y']
        pos = result['align_position']
        for i in np.arange(len(species)):
            if key == species[i]:
                if str(verif_motifs_dict[key]) == str(pos[i]):
                    orig[i] = 'yes'
    result['motif?'] = orig
    result = result.drop(columns=['species_x'])
    return result
                    
                    
                    

In [55]:
def raw_string(spec, motif_length, no_thresh_path, motif_key, path_to_raw):
    """ spec: The region number ex: 11048
        length: The length of the sequence that we want
        path: The path to the folder with the thresholded and non thresholded directories
        
        Returns a csv saved under data/output/full_raw_motif_extraction/*spec*
        """
    
    result = prep_table(spec, motif_key, no_thresh_path)
    num_spec = spec.split('.')[0]
    if num_spec[0] == "_":
        num_spec = num_spec[1:]
    print(path_to_raw)
    print("num_spec:" + num_spec)
    print("spec:" + spec)
    spec_path = path_to_raw + "/outlier_rm_with_length_"+str(num_spec)+".fa"
    record_dict = SeqIO.to_dict(SeqIO.parse(spec_path, "fasta"))
    sequences = []
    before = []
    
    after = []
    length = motif_length
    for index, row in result.iterrows():
        speci = row['species_y']
        #print(spec)
        pos = row['raw_position']
        #print(pos)
        strand = row['strand']
        #print(strand)
        seq = record_dict[speci]
        if strand == 'negative':
            sequences.append(getNegative(seqToString(seq[pos:pos + 6])))
            before.append(getNegative(seqToString(seq[pos - length:pos])))
            after.append(getNegative(seqToString(seq[pos + 6:pos + 6 + length])))
        else:
            sequences.append(seqToString(seq[pos:pos + 6]))
            before.append(seqToString(seq[pos - length:pos]))
            after.append(seqToString(seq[pos + 6:pos + 6 + length]))
    # print(sequences)
    # print(len(sequences))
    # print(sequences.index('TAAGCC'))
    result['raw_seq'] = np.array(sequences)
    result['before_seq'] = np.array(before)
    result['after_seq'] = np.array(after)
    result
    if os.path.exists("/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/output") == False:
        os.mkdir("/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/output")
    if os.path.exists("/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/output/" + str(motif_key)) == False:
        os.mkdir(os.path.join("/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/output/", str(motif_key)))
    result.to_csv("/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/output/" + str(motif_key) + "/" + str(num_spec) +  "_final_raw.fa.csv")

In [56]:
path = "/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd"
path_to_raw = "/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only"

In [67]:
for motif in os.listdir(path):
    motif_key = motif[:3]
    spec = motif.split('_')[-1].split('.')[0]
    print(motif)
    print(os.path.isfile(os.path.join(path, motif)))
    if not os.path.isfile("/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/output/"+ str(motif_key) + "/" + str(spec) + "_final_raw.fa.csv"):
        raw_string(motif, getMotifLength('bcd'), os.path.join(path, motif), "bcd", path_to_raw)

VT43710.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT43710.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT43710
spec:VT43710.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT22205.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT22205.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT22205
spec:VT22205.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT23746.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT23746.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT2374

/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT27740
spec:VT27740.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT58869.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT58869.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT58869
spec:VT58869.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT40174.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT40174.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT40174
spec:VT40174.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT28323.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_

/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT48342
spec:VT48342.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT11075.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT11075.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT11075
spec:VT11075.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT33038.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT33038.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT33038
spec:VT33038.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT43927.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_

/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT47168
spec:VT47168.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT46130.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT46130.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT46130
spec:VT46130.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT56362.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT56362.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT56362
spec:VT56362.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT27670.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_

/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT39960
spec:VT39960.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT62770.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT62770.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT62770
spec:VT62770.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT34271.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT34271.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT34271
spec:VT34271.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT33646.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_

...

/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT39513
spec:VT39513.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT29741.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT29741.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT29741
spec:VT29741.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT18474.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT18474.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT18474
spec:VT18474.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT26839.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_

/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT46246
spec:VT46246.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT62864.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT62864.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT62864
spec:VT62864.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT40167.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT40167.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT40167
spec:VT40167.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT16466.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_

/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT5531
spec:_VT5531.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT19529.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT19529.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT19529
spec:VT19529.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT21810.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT21810.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT21810
spec:VT21810.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT16604.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_0

/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT31180
spec:VT31180.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT16467.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT16467.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT16467
spec:VT16467.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT17685.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT17685.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT17685
spec:VT17685.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT46012.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_

/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT22220
spec:VT22220.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT63311.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/VT63311.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT63311
spec:VT63311.fa_MA0212.1_bcd.jaspar_bashoutput.csv
_VT2849.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_06/bcd/_VT2849.fa_MA0212.1_bcd.jaspar_bashoutput.csv
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/3.24_species_only
num_spec:VT2849
spec:_VT2849.fa_MA0212.1_bcd.jaspar_bashoutput.csv
VT24101.fa_MA0212.1_bcd.jaspar_bashoutput.csv
True
is no thresh path
True
/Volumes/ciera_1/analysis/2.re-running_whole_pipeline/data/jaspar_redo_2019_09_0

In [58]:
s = "outlier_rm_with_length_VT0809.fa"

In [64]:
s.split('_')[-1].split('.')[0]

'VT0809'